In [977]:
import schedule
import time
import requests
import pandas as pd
import csv
from datetime import datetime, timedelta, date
import pyodbc
from sqlalchemy import create_engine
import mysql.connector

In [978]:
db_config = {
    'host': 'Localhost',
    'user': 'root',
    'password': '8november2002',
    'database': 'bhavDB',
}
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()


In [979]:
def redEXTRA(date,symbol):
    # high(last ten days)-low (last ten days)/low(last ten days)
    current_date=datetime.strptime(date,"%d%m%Y") #25-01-2024
    current_date-=timedelta(days=1)
    query=f"SELECT * FROM StockData2024 WHERE DATE1 = '{current_date}' and SYMBOL = '{symbol}'"
    cursor.execute(query)
    column_names = [desc[0] for desc in cursor.description]
    
    counter=0
    result= cursor.fetchall()
    counter+=1
    df_current = pd.DataFrame(result, columns=column_names)

    today = df_current[df_current['SYMBOL'] == symbol]
    today = today.iloc[0]

    high=today['HIGH_PRICE']
    low=today['LOW_PRICE']

    while(True):
        query=f"SELECT * FROM StockData2024 WHERE DATE1 = '{current_date}' and SYMBOL = '{symbol}'"
        cursor.execute(query)
        result= cursor.fetchall()
        if(len(result)!=0):

            df_current = pd.DataFrame(result, columns=column_names)
            today = df_current[df_current['SYMBOL'] == symbol]
            if today.empty:
                continue
            today = today.iloc[0]
            if(today['HIGH_PRICE']>high):
                high=today['HIGH_PRICE']
            if(today['LOW_PRICE']<low):
                low=today['LOW_PRICE']
            
            counter+=1
        current_date-=timedelta(days=1)
        if counter==10:
            break
    
    percent=(high-low)/high
    if(percent>=.1):
        return True
    return False

In [980]:
def redEXTRA2(date, symbol):
    current_date=datetime.strptime(date,"%d%m%Y")
    c=0
    while(c!=2):
        query=f"SELECT * FROM StockData2024 WHERE DATE1 = '{current_date}' and SYMBOL = '{symbol}'"
        cursor.execute(query)
        column_names = [desc[0] for desc in cursor.description]
        result= cursor.fetchall()
        if(len(result)!=0):
            c+=1
        current_date-=timedelta(days=1)
        if(c==2):
            while(True):
                query=f"SELECT * FROM StockData2024 WHERE DATE1 = '{current_date}' and SYMBOL = '{symbol}'"
                cursor.execute(query)
                column_names = [desc[0] for desc in cursor.description]
                result= cursor.fetchall()
                if(len(result)!=0):
                    break
                current_date-=timedelta(days=1)
    query=f"SELECT * FROM StockData2024 WHERE DATE1 = '{current_date}' and SYMBOL = '{symbol}'"
    cursor.execute(query)
    result= cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df_red = pd.DataFrame(result, columns=column_names)
    red_data = df_red[df_red['SYMBOL'] == symbol]
    if red_data.empty:
        return False
    red_data = red_data.iloc[0]

    red_body_length=abs(red_data['OPEN_PRICE']-red_data['CLOSE_PRICE'])
    red_candle_length=red_data['HIGH_PRICE']-red_data['LOW_PRICE']

    #  current_date -> red candle
    counter=0
    while(True):
        current_date-=timedelta(days=1)
        query=f"SELECT * FROM StockData2024 WHERE DATE1 = '{current_date}' and SYMBOL = '{symbol}'"
        cursor.execute(query)
        result= cursor.fetchall()
        if(len(result)!=0):
            counter+=1
            data = pd.DataFrame(result, columns=column_names)
            df_data = data[data['SYMBOL'] == symbol]
            if df_data.empty:
                continue
            df_data = df_data.iloc[0]
            curr_body_length=abs(df_data['OPEN_PRICE']-df_data['CLOSE_PRICE'])
            curr_candle_length=df_data['HIGH_PRICE']-df_data['LOW_PRICE']
            if(curr_body_length>red_body_length or curr_candle_length>red_candle_length):
                return False
        if(counter==8):
            break
    return True

    

In [981]:
import pandas as pd
def redSPY(df_today, df_prev, df_prev2, df_prev3, date):
    
    symbols = df_today['SYMBOL'].unique().tolist()

    for symbol in symbols:
        today = df_today[df_today['SYMBOL'] == symbol]
        prev_day = df_prev[df_prev['SYMBOL'] == symbol]
        prev_day2 = df_prev2[df_prev2['SYMBOL'] == symbol]
        prev_day3 = df_prev3[df_prev3['SYMBOL'] == symbol]

        # Check if any of the filtered DataFrames is empty
        if today.empty or prev_day.empty or prev_day2.empty or prev_day3.empty:
            continue

        today = today.iloc[0]
        second_day = prev_day.iloc[0]
        first_day = prev_day2.iloc[0]
        day_before_red =prev_day3.iloc[0]

        # Apply condition to find colors of all three days candles
        second_day_color = 'red' if second_day['OPEN_PRICE'] > second_day['CLOSE_PRICE'] else 'green'
        first_day_color = 'green' if first_day['OPEN_PRICE'] < first_day['CLOSE_PRICE'] else 'red'
        today_color = 'green' if today['OPEN_PRICE'] < today['CLOSE_PRICE'] else 'red'

        # Condition check
        condition_met = (
            first_day_color == 'green' and
            second_day_color == 'red' and
            today_color == 'red' and
            second_day['LOW_PRICE'] > first_day['OPEN_PRICE'] and
            second_day['HIGH_PRICE'] < first_day['CLOSE_PRICE'] and
            today['CLOSE_PRICE'] < first_day['LOW_PRICE'] and
            redEXTRA(date,symbol) and 
            redEXTRA2(date, symbol)
        )
        # print(f"For Symbol: {symbol, condition_met}")
        # Check if the condition is met and proceed
        if condition_met:
            stop_loss = first_day['HIGH_PRICE']
            red_body_percent=100*(first_day['OPEN_PRICE']-first_day['CLOSE_PRICE'])/(first_day['HIGH_PRICE']-first_day['LOW_PRICE'])
            # also print 100*(volume of red candle-volume of previous candle of red candle)/volume of previous candle
            volume=100*(first_day['TTL_TRD_QNTY']-day_before_red['TTL_TRD_QNTY'])/day_before_red['TTL_TRD_QNTY']

            print(f"For Symbol: {symbol}")
            print(f"Stop Loss: {stop_loss}")
            print()
            filename = 'redSPY.csv'
            with open(filename, 'a', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                date_object = datetime.strptime(date, '%d%m%Y')
                csvwriter.writerows([[symbol,date_object.strftime('%d-%b-%Y'),stop_loss,volume,red_body_percent]])

In [982]:
data=[]
date="25062024"
current_date=datetime.strptime(date,"%d%m%Y") #25-01-2024
query=f"SELECT * FROM StockData2024 WHERE DATE1 = '{current_date}'"
cursor.execute(query)
result= cursor.fetchall()
if(len(result)!=0):
    counter=0
    data.append(result)
    counter+=1
    print(f"Open {current_date}")
    current_date-=timedelta(days=1)
    while(True):
        #Check response and append data
        query=f"SELECT * FROM StockData2024 WHERE DATE1 = '{current_date}'"
        cursor.execute(query)
        result= cursor.fetchall()
        if(len(result)!=0):
            data.append(result)
            counter+=1
            print(f"Open {current_date}")
        else:
            print(f"Close {current_date}")
        current_date-=timedelta(days=1)
        #Break Loop
        if(counter==4):
            break
    print(data)
    column_names = [desc[0] for desc in cursor.description]
    print(column_names)

    df_today = pd.DataFrame(data[0], columns=column_names)
    df_prev = pd.DataFrame(data[1], columns=column_names)
    df_prev2 = pd.DataFrame(data[2], columns=column_names)
    df_prev3 = pd.DataFrame(data[3], columns=column_names)

    redSPY(df_today, df_prev, df_prev2, df_prev3,date)
else:
    print("Market Closed")

Market Closed


In [983]:
connection.commit()
cursor.close()
connection.close()